In [1]:
from dask.distributed import Client, LocalCluster
import dask_geopandas as dask_geopandas
import dask.dataframe as dd
import dask.bag as db
import geopandas
import pygeos
import numpy as np
import pandas as pd
import momepy

import os

from sqlalchemy import create_engine
from libpysal.weights import Queen

from momepy_utils import street_profile

/opt/conda/lib/python3.7/site-packages/pyarrow/compat.py:25: FutureWarning: pyarrow.compat has been deprecated and will be removed in a future release
  "future release", FutureWarning)


In [2]:
client = Client(LocalCluster(n_workers=12))
client

Client Scheduler: tcp://127.0.0.1:39777 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 36 Memory: 84.28 GB


In [3]:
cross_chunk = pd.read_parquet('../../urbangrammar_samba/spatial_signatures/cross-chunk_indices.pq')
chunks = geopandas.read_parquet('../../urbangrammar_samba/spatial_signatures/local_auth_chunks.pq')

In [4]:
user = os.environ.get('DB_USER')
pwd = os.environ.get('DB_PWD')
host = os.environ.get('DB_HOST')
port = os.environ.get('DB_PORT')

db_connection_url = f"postgres+psycopg2://{user}:{pwd}@{host}:{port}/built_env"

In [5]:
def measure(chunk_id):
    print(chunk_id)
    blg = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/buildings/blg_{chunk_id}.pq")
    tess = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/tessellation/tess_{chunk_id}.pq")
    orig_ids = tess.uID.copy()
    
    # expand across the chunk boundary
    additional_tess = []
    addtional_blg = []
    add_tess_data = []
    for chunk, inds in cross_chunk.loc[1].indices.iteritems():
        add_tess = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/tessellation/tess_{chunk}.pq").iloc[inds]
        additional_tess.append(add_tess)
        add_blg = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/buildings/blg_{chunk}.pq")
        addtional_blg.append(add_blg[add_blg.uID.isin(add_tess.uID)])
        tess_data = pd.read_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/cells/cells_{chunk}.pq")
        add_tess_data.append(tess_data[tess_data.uID.isin(add_tess.uID)])
    
    add_tess_data.append(pd.read_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/cells/cells_{chunk_id}.pq"))
    additional_tess.append(tess)
    tess = pd.concat(additional_tess)
    addtional_blg.append(blg)
    blg = pd.concat(addtional_blg)
    
    blg = blg.rename_geometry('buildings')
    tess = tess.rename_geometry('tessellation')

    df = tess.merge(blg, on='uID', how='left')
    df = df.merge(pd.concat(add_tess_data).drop(columns='enclosureID'), on='uID', how='left')
    
    # make 3d geometry 2d
    coords = pygeos.get_coordinates(df.tessellation.values.data)
    counts = pygeos.get_num_coordinates(df.tessellation.values.data)
    df['tessellation'] = geopandas.GeoSeries([pygeos.polygons(c) for c in np.split(coords, np.cumsum(counts)[:-1])], crs=df.tessellation.crs)

    w = Queen.from_dataframe(df, geom_col='tessellation')

    df['ix'] = range(len(df))


    def alignment(x, orientation):
        orientations = df[orientation].iloc[w.neighbors[x]]
        return abs(orientations - df[orientation].iloc[x]).mean()


    df['mtbAli'] = df.ix.apply(alignment, args=('stbOri',))


    def neighbor_distance(x):
        geom = df.buildings.iloc[x]
        if geom is None:
            return np.nan
        return df.buildings.iloc[w.neighbors[x]].distance(df.buildings.iloc[x]).mean()


    df['mtbNDi'] = df.ix.apply(neighbor_distance)

    df['mtcWNe'] = df.ix.apply(lambda x: w.cardinalities[x]) / df.tessellation.length


    def area_covered(x, area):
        neighbours = [x]
        neighbours += w.neighbors[x]

        return df[area].iloc[neighbours].sum()


    df['mdcAre'] = df.ix.apply(area_covered, args=('sdcAre',))

    w3 = momepy.sw_high(k=3, weights=w)

    # define adjacency list from lipysal
    adj_list = w.to_adjlist(remove_symmetric=True)
    adj_list["distance"] = (
        df.buildings.iloc[adj_list.focal]
        .reset_index(drop=True)
        .distance(df.buildings.iloc[adj_list.neighbor].reset_index(drop=True))
    )
    adj_list = adj_list.set_index(['focal', 'neighbor'])


    def mean_interbuilding_distance(x):
        neighbours = [x]
        neighbours += w3.neighbors[x]
        return adj_list.distance.loc[neighbours, neighbours].mean()


    df['ltbIBD'] = df.ix.apply(mean_interbuilding_distance)


    def weighted_reached_enclosures(x, area, enclosure_id):
        neighbours = [x]
        neighbours += w3.neighbors[x]

        vicinity = df[[area, enclosure_id]].iloc[neighbours]

        return vicinity[enclosure_id].unique().shape[0] / vicinity[area].sum()

    df['ltcWRE'] = df.ix.apply(weighted_reached_enclosures, args=('sdcAre', 'enclosureID'))
    
    df.drop(columns=['buildings', 'tessellation']).to_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/cells/cells_{chunk_id}.pq")
    
    
    chunk_area = chunks.geometry.iloc[chunk_id].buffer(5000)
    engine = create_engine(db_connection_url)
    sql = f"SELECT * FROM openroads_200803_topological WHERE ST_Intersects(geometry, ST_GeomFromText('{chunk_area.wkt}',27700))"
    streets = geopandas.read_postgis(sql, engine, geom_col='geometry')
    
    sp = street_profile(streets, blg)
    streets['sdsSPW'] = sp[0]
    streets['sdsSWD'] = sp[1]
    streets['sdsSPO'] = sp[2]
    
    streets['sdsLen'] = streets.length
    streets['sssLin'] = momepy.Linearity(streets).series
    
    G = momepy.gdf_to_nx(streets)
    G = momepy.node_degree(G)
    G = momepy.subgraph(
        G,
        radius=5,
        meshedness=True,
        cds_length=False,
        mode="sum",
        degree="degree",
        length="mm_len",
        mean_node_degree=False,
        proportion={0: True, 3: True, 4: True},
        cyclomatic=False,
        edge_node_ratio=False,
        gamma=False,
        local_closeness=True,
        closeness_weight="mm_len",
        verbose=False
    )
    G = momepy.cds_length(G, radius=3, name="ldsCDL", verbose=False)
    G = momepy.clustering(G, name="xcnSCl")
    G = momepy.mean_node_dist(G, name="mtdMDi", verbose=False)
    
    nodes, edges, sw = momepy.nx_to_gdf(G, spatial_weights=True)
    
    edges_w3 = momepy.sw_high(k=3, gdf=edges)
    
    edges["ldsMSL"] = momepy.SegmentsLength(edges, spatial_weights=edges_w3, mean=True, verbose=False).series
    
    nodes_w5 = momepy.sw_high(k=5, weights=sw)
    
    nodes["lddNDe"] = momepy.NodeDensity(nodes, edges, nodes_w5, verbose=False).series
    
    nodes["linWID"] = momepy.NodeDensity(nodes, edges, nodes_w5, weighted=True, node_degree="degree", verbose=False).series
    
    edges.to_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/edges/edges_{chunk_id}.pq")
    nodes.to_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/nodes/nodes_{chunk_id}.pq")
    return True

In [12]:
%time measure(0)

/opt/conda/lib/python3.7/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 23 disconnected components.
 There is 1 island with id: 128971.
  warnings.warn(message)
/opt/conda/lib/python3.7/site-packages/libpysal/weights/weights.py:309: UserWarning: {} islands in this weights matrix. Conversion to an adjacency list will drop these observations!
  "{} islands in this weights matrix. Conversion to an "
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/jovyan/work/spatial_signatures/morphometrics/momepy_utils.py:469: RuntimeWarning: invalid value encountered in long_scalars
  openness.append(np.isnan(s).sum() / (f).sum())
/opt/conda/lib/python3.7/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 49 disconnected components.
  warnings.warn(message)


CPU times: user 1h 20min 23s, sys: 9.09 s, total: 1h 20min 32s
Wall time: 1h 20min 35s


True

In [ ]:
%%time
bag = db.from_sequence(list(range(103)), npartitions=12)
new = bag.map(measure).compute()